Import the necessary packages and API key.

In [145]:
!pwd

/Users/jmirabito/Desktop/Learn.co/mod_1_movie_project/2020-movie-analysis


In [10]:
!git branch

* JamBranch
  master


In [136]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [139]:
pip install "requests[security]"

Note: you may need to restart the kernel to use updated packages.


In [89]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [84]:
keys = get_keys("/Users/jmirabito/Desktop/Learn.co/.secret/tmdb_api")

api_key = keys['api_key']

In [13]:
import requests

In order to use the TMDB API, we first import a list of 73,856 unique IMDb movie IDs so that we can use those to find the necessary information from the TMDB database. We pull this information in from our cleaned IMDb data file.

In [50]:
imdb_id = pd.read_csv('/Users/jmirabito/Desktop/Learn.co/mod_1_movie_project/2020-movie-analysis/Data/imdb.basics_and_ratings_nonulls.csv')
imdb_id.head()

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt1612608,The Italian,L'Italien,2010,102.0,"Comedy,Drama",5.8,1360
1,tt4874854,The Legend of Zelda: The Triforce Prophecy,The Legend of Zelda: The Triforce Prophecy,2010,120.0,"Adventure,Fantasy,Romance",4.8,8
2,tt1718842,Nakalnya anak muda,Nakalnya anak muda,2010,75.0,"Horror,Thriller",4.0,12
3,tt1645134,Sandcastle,Sandcastle,2010,91.0,Drama,6.3,116
4,tt1645129,Planzet,Puranzetto,2010,53.0,"Action,Animation,Sci-Fi",4.6,290


We then pull out the relevant column containing our IMDb id.

In [82]:
external_id = list(imdb_id['title_id'])
len(external_id)

73856

Using the API string request format provided in the TMDB API documentation, we tested the code by inserting a random value from the IMDb id list ('external_id'). Our output from data.json() is a dictionary, so we indexed our way into the dictionary to extract the relevant information - in this case the TMDB id.

In [64]:
request_string = 'https://api.themoviedb.org/3/find/{}?api_key={}&language=en-US&external_source=imdb_id'.format(external_id[5436], api_key)
data = requests.get(request_string)
data.json()['movie_results'][0]['id']


513784

In [83]:
# tmdb_id =[] 
for item in external_id:
    request_string = 'https://api.themoviedb.org/3/find/{}?api_key={}&language=en-US&external_source=imdb_id'.format(item, api_key)
    data = requests.get(request_string)
    data.json()['movie_results']
    if data.json()['movie_results'] == []:
        continue
    else:
        local_id = data.json()['movie_results'][0]['id']
        tmdb_id.append(local_id)



605343

In [86]:
len(tmdb_id)

59429

In [142]:
# tmdb_dicts = []
for i in tqdm(range(37644, len(tmdb_id))):
    request_string = 'https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US'.format(tmdb_id[i], api_key)
    data = requests.get(request_string)
    if data.ok:
        movie_dict = {'tmdb_id': data.json()['id'],
                      'imdb_id': data.json()['imdb_id'],
                      'title': data.json()['title'],
                      'genre': [data.json()['genres'][i]['name'] for i in list(range(0, len(data.json()['genres'])))],
                      'studio': [data.json()['production_companies'][i]['name'] for i in list(range(0, len(data.json()['production_companies'])))],
                      'budget': data.json()['budget'],
                      'revenue': data.json()['revenue'],
                      'tmdb_rating': data.json()['vote_average'],
                      'tmdb_num_votes': data.json()['vote_count']
                     }
        tmdb_dicts.append(movie_dict)
    else:
        movie_dict = {'tmdb_id': 'null',
                      'imdb_id': 'null',
                      'title': 'null',
                      'genre': 'null',
                      'studio': 'null',
                      'budget': 'null',
                      'revenue': 'null',
                      'tmdb_rating': 'null',
                      'tmdb_num_votes': 'null'
                     }
        tmdb_dicts.append(movie_dict)

    
    

100%|██████████| 21785/21785 [52:30<00:00,  6.92it/s]  


In [143]:
len(tmdb_dicts)

59429

In [144]:
tmdb_info = pd.DataFrame.from_dict(tmdb_dicts)
tmdb_info.columns = tmdb_dicts[0].keys()
tmdb_info.head()

,tmdb_id,imdb_id,title,genre,studio,budget,revenue,tmdb_rating,tmdb_num_votes
0,44155,tt1612608,The Italian,[Comedy],"[Eskwad, Pathé!, M6 Films, Canal+]",0,0,5.3,144
1,642129,tt1718842,Nakalnya Anak Muda,"[Horror, Drama, Thriller]",[Mitra Pictures],0,0,0,0
2,69237,tt1645134,Sandcastle,[Drama],[],0,0,6,4
3,46249,tt1645129,Planzet,"[Animation, Action, Science Fiction]","[CoMix Wave Films, Media Factory]",0,0,6.2,5
4,347635,tt1645115,Naica,[],[],0,0,0,0


In [150]:
tmdb_info.to_csv('Data/tmdb_api_data.csv')